In [1]:
import pandas as pd
import numpy as np

### Station List Data : data was pulled to get longitude and latitude to the weather data which only had corresponding stations

In [5]:
f = open('./data/stations.txt','r')
var_list = f.read().split('\n')

In [6]:
new_list = []
for x in range(len(var_list)):
    new_list.append(var_list[x].split())

In [7]:
key_df = pd.DataFrame(new_list)

In [8]:
key_df = key_df.iloc[:,:5]

In [9]:
key_df.rename(columns = {0:'station_id',1:'latitude',2:'longitude',3:'elevation',4:'state_real'},inplace = True)

### Cleaning Weather Data 

Weather data sets only had 2 columns. The rest was configured with rows of the same day but with different features. This had to be changed and altered to be configured for columned data. Ultimately all years of weather data turned out to be 30 million rows.

In [10]:
def clean_weather(file_name):
    df = pd.read_csv(f'./data/raw_weather/{file_name}', header = None)
    #dropping unused columns
    df = df.drop(columns = [4,5,6,7])
    # seperating data from ditinctions in rows min and max temperature
    df = df.loc[df[2].isin(['TMIN','TMAX'])]
    df_tmax = df.loc[df[2]=='TMAX']
    df_tmin = df.loc[df[2]=='TMIN']
    #adjusting row data to now be columns
    df_tmax.rename(columns = {0:'station_id',1:'date',2:2,3:'tmax'}, inplace = True)
    df_tmin.rename(columns = {0:'station_id',1:'date',2:2,3:'tmin'}, inplace = True)
    df_tmax = df_tmax.drop(columns = 2)
    df_tmin = df_tmin.drop(columns = 2)
    #changing data to datetime
    df_tmin['date'] = pd.to_datetime(df_tmin['date'], format='%Y%m%d')
    df_tmax['date'] = pd.to_datetime(df_tmax['date'], format='%Y%m%d')
    df_total = pd.merge(df_tmin,df_tmax, on = ['station_id','date'])
    final_df = pd.merge(left = df_total,right = key_df,how = 'inner',on = 'station_id')
    return final_df

In [11]:
def clean_weather_to_csv(file_name):
    df = clean_weather(file_name)

    return df.to_csv(f'./data/clean_weather/true_cleaned{file_name[:4]}.csv',index = False)

In [12]:
#running all years
for x in range(2000,2019):
    clean_weather_to_csv(f'{x}.csv.gz')
    print(f'{x} csv done')


2000 csv done
2001 csv done
2002 csv done
2003 csv done
2004 csv done
2005 csv done
2006 csv done
2007 csv done
2008 csv done
2009 csv done
2010 csv done
2011 csv done
2012 csv done
2013 csv done
2014 csv done
2015 csv done


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2016 csv done
2017 csv done
2018 csv done


In [13]:
df_list_00_18 = [pd.read_csv(f'./data/clean_weather/true_cleaned{x}.csv') for x in range(2000,2019)]

df = pd.concat(df_list_00_18)

df.to_csv('./data/true_total_weather.csv', index = False)

In total this is 5 gb of data.

### Combing Fire and Weather data sets.

This adds the temperatures and locations for the fires with station ids.

In [49]:
fdf = pd.read_csv('./data/fire.csv')

In [17]:
wdf = pd.read_csv('./data/true_total_weather.csv')

In [50]:
fdf['state_real'] = fdf['Fire_ID,C,254'].map(lambda x : x[:2])

In [51]:
fdf['IG_Date,D'] = pd.to_datetime(fdf['IG_Date,D'])

In [52]:
fdf.rename(columns = {'IG_Date,D':'date'},inplace = True)

In [21]:
wdf['date'] = pd.to_datetime(wdf['date'])

In [22]:
wdf['tmin'] = wdf['tmin'].map(lambda x : x * .1)

In [23]:
wdf['tmax'] = wdf['tmax'].map(lambda x : x * .1)

In [24]:
wdf['temp_avg'] =( wdf['tmax'] + wdf['tmin'])/2

In [25]:
wdf['daily_state_temp'] = wdf.groupby(['date','state_real'])[['temp_avg']].transform('mean')

In [27]:
wdf.to_csv('./data/final_wdf.csv', index = False)

In [39]:
wdf = wdf.drop_duplicates(['daily_state_temp','date','state_real'])

In [41]:
wdf.reset_index(drop = 'index',inplace = True)

In [43]:
wdf['daily_state_temp']=wdf['daily_state_temp'].map(lambda x : (x * 9/5) + 32)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
wdf = wdf[['daily_state_temp','date','state_real']]

In [53]:
a_fdf = pd.merge(left = fdf, right = wdf, on =['state_real','date'])

In [56]:
a_fdf.to_csv('./data/adjusted_fire.csv',index = False)